In [1]:
import pandas as pd
import os
import calendar
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport
import numpy as np

In [8]:
def wrangle_file(file_path, year_of_file):
    ## Read the files
    df = pd.read_parquet(file_path, engine='pyarrow')

    ## Data Cleansing
    columns_drop = [
        'VendorID', 'airport_fee', 'RatecodeID', 'store_and_fwd_flag', 'extra'
        , 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge'
    ]
    df = df.drop(columns=columns_drop)
    df = df.dropna()

    # Keep only data reported for a span of a year before the year of the file
    df = df[df.tpep_pickup_datetime.dt.year.between(year_of_file-1, year_of_file, inclusive='both')]

    # Dropoff should be after pickup
    df = df[df['total_amount']>0]
    df = df[(df['passenger_count']>0) & (df['passenger_count']<7)]
    df = df[(df['trip_distance']>0) & (df['trip_distance']<=50)]
    df = df[(df['fare_amount']>0)]
    df = df[(df['tip_amount']<=100)]
    

    ## Transformations
    # Get the trip duration and filter to trips with duration greater to zero
    df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.seconds
    df = df[(df['trip_duration']>0)]

    # Round observations to the level of hour
    df['pickup_datetime'] = df['tpep_pickup_datetime'].dt.floor('h')

    # Add Pickup & Dropoff Zone Data
    df['pickup_from'] = np.select( [
            df['PULocationID'].isin([1, 132, 138]) # Location is one of the three NYC airports
            , ~df['PULocationID'].isin([1, 132, 138]) # Location is NOT one of the three NYC airports
        ]
        , [
            'Airport'
            , 'Other'
        ],
        default='Unknown'
    )

    df['dropoff_at'] = np.select( [
            df['DOLocationID'].isin([1, 132, 138]) # Location is one of the three NYC airports
            , ~df['DOLocationID'].isin([1, 132, 138]) # Location is NOT one of the three NYC airports
        ]
        , [
            'Airport'
            , 'Other'
        ],
        default='Unknown'
    )
    
    # Dropped this as I didn't really need the join for the analysis I was doing, just the ID's of the airports
    # df_zones.columns = ['pickup_id', 'pickup_borough', 'pickup_zone', 'pickup_service_zone']
    # df = df.merge(df_zones, how='left', left_on='PULocationID', right_on='pickup_id')
    # df_zones.columns = ['dropoff_id', 'dropoff_borough', 'dropoff_zone', 'dropoff_service_zone']
    # df = df.merge(df_zones, how='left', left_on='DOLocationID', right_on='dropoff_id')

    df['trips'] = 1

    # Aggregate the data at the hour level
    df_hourly = df.groupby([
        'pickup_datetime', 'payment_type', 'pickup_from', 'dropoff_at'
    ]).agg({
        'passenger_count': ['mean']
        , 'trip_distance': ['mean']
        , 'fare_amount': ['mean']
        , 'tip_amount': ['mean']
        , 'trip_duration': ['mean']
        , 'total_amount': ['mean']
        , 'trips': ['sum']
    })

    df_hourly = df_hourly.reset_index()

    df_hourly.columns = [
        'pickup_datetime'
        , 'payment_type'
        , 'pickup_from'
        , 'dropoff_at'
        , 'passenger_count'
        , 'trip_distance'
        , 'fare_amount'
        , 'tip_amount'
        , 'trip_duration'
        , 'total_amount'
        , 'trips'
    ]

    # Get the floor of the average of passengers
    df_hourly['passenger_count'] = df_hourly['passenger_count'].apply(np.floor).astype('int')

    # New Categorical Variables
    df_hourly['congestion_category'] = np.select( [
            df_hourly['pickup_datetime'].dt.hour.between(0, 5, inclusive='left')
            , df_hourly['pickup_datetime'].dt.hour.between(5, 8, inclusive='left')
            , df_hourly['pickup_datetime'].dt.hour.between(8, 13, inclusive='left')
            , df_hourly['pickup_datetime'].dt.hour.between(13, 17, inclusive='left')
            , df_hourly['pickup_datetime'].dt.hour.between(17, 24, inclusive='left' ) 
        ]
        , [
            'After Midnight Congestion'
            , 'Early Morning Congestion'
            , 'Leading to Noon Congestion'
            , 'Afternoon Congestion'
            , 'Evening Congestion'
        ],
        default='Unknown'
    )

    df_hourly['day_category'] = np.select( [
            df_hourly['pickup_datetime'].dt.hour.between(0, 6, inclusive='left')
            , df_hourly['pickup_datetime'].dt.hour.between(6, 12, inclusive='left')
            , df_hourly['pickup_datetime'].dt.hour.between(12, 18, inclusive='left')
            , df_hourly['pickup_datetime'].dt.hour.between(18, 24, inclusive='left')
        ]
        , [
            'Early Morning'
            , 'Late Morning'
            , 'Afternoon'
            , 'Evening'
        ],
        default='Unknown'
    )

    df_hourly['payment_type'] = np.select( [
            df_hourly['payment_type'] == 1
            , df_hourly['payment_type'] == 2
            , df_hourly['payment_type'] == 3
            , df_hourly['payment_type'] == 4
            , df_hourly['payment_type'] == 5
            , df_hourly['payment_type'] == 6
        ]
        , [
            'Credit Card'
            , 'Cash'
            , 'No Charge'
            , 'Dispute'
            , 'Unknown'
            , 'Voided Trip'
        ],
        default='Unknown'
    )

    df_hourly['source_file'] = file_path

    return df_hourly

In [9]:
# ## Read the files
df_zones = pd.read_csv( r"F:\BFD Project Data\Complementary\taxi+_zone_lookup.csv")

file_path=r"F:\BFD Project Data\Raw\2017\yellow_tripdata_2017-04.parquet"
wrangled_df = wrangle_file(file_path, year_of_file=2017)
wrangled_df

,pickup_datetime,payment_type,pickup_from,dropoff_at,passenger_count,trip_distance,fare_amount,tip_amount,trip_duration,total_amount,trips,congestion_category,day_category,source_file
0,2017-04-01 00:00:00,Credit Card,Airport,Airport,1,15.106667,45.611111,7.346667,1461.444444,57.310000,9,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...
1,2017-04-01 00:00:00,Credit Card,Airport,Other,1,13.778093,39.997881,7.520862,1800.522599,51.650890,708,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...
2,2017-04-01 00:00:00,Credit Card,Other,Airport,1,17.375000,53.125000,5.387500,2685.000000,62.262500,4,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...
3,2017-04-01 00:00:00,Credit Card,Other,Other,1,2.919678,12.786823,2.538089,976.162297,16.747163,11596,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...
4,2017-04-01 00:00:00,Cash,Airport,Airport,2,3.304375,14.125000,0.000000,454.375000,15.393750,16,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9387,2017-04-30 23:00:00,No Charge,Airport,Airport,1,0.100000,2.500000,0.000000,18.000000,3.800000,1,Evening Congestion,Evening,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...
9388,2017-04-30 23:00:00,No Charge,Airport,Other,1,12.516667,41.750000,0.000000,1550.500000,44.970000,6,Evening Congestion,Evening,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...
9389,2017-04-30 23:00:00,No Charge,Other,Other,1,4.962222,18.544444,0.000000,977.377778,19.800000,45,Evening Congestion,Evening,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...
9390,2017-04-30 23:00:00,Dispute,Airport,Other,1,11.333333,33.166667,0.000000,1234.666667,40.060000,3,Evening Congestion,Evening,F:\BFD Project Data\Raw\2017\yellow_tripdata_2...


In [5]:
## Read the files
df_zones = pd.read_csv( r"F:\BFD Project Data\Complementary\taxi+_zone_lookup.csv")

file_path=r'F:\BFD Project Data\Raw'
df_hourly_combined = pd.DataFrame()

for year in os.listdir(file_path):
    for file in os.listdir('{0}/{1}'.format(file_path, year)):
        path = '{0}\{1}\{2}'.format(file_path, year, file)
        df_wrangled = wrangle_file(path, year_of_file=int(year))
        df_hourly_combined = pd.concat([df_hourly_combined, df_wrangled])

# df_keep = df_hourly_combined.copy()

In [9]:
## Final Processing
# Get only trips with a maximum of $150 as the total_amout paid
df_hourly_combined = df_hourly_combined[df_hourly_combined['total_amount'].between(0, 150, inclusive='right')]

# Get only trips with a maximum duration of an hour and 30 mins
df_hourly_combined = df_hourly_combined[df_hourly_combined['trip_duration'].between(0, 5400, inclusive='right')]

# After flooring the passenger count, remove trips with no passengers
df_hourly_combined = df_hourly_combined[df_hourly_combined['passenger_count'] > 0]

# Sort by pickup_datetime
df_hourly_combined = df_hourly_combined.sort_values('pickup_datetime')

## Write to File
df_hourly_combined['id'] = [*range(0, df_hourly_combined.shape[0], 1)]
df_hourly_combined.set_index(df_hourly_combined['id'])
path_file_output = 'F:\BFD Project Data\Processed\yellow_taxi_data_hourly_v6.csv'
df_hourly_combined.to_csv(path_file_output, index=False)

In [2]:
## Post EDA Processing
path_file_output = 'F:\BFD Project Data\Processed\yellow_taxi_data_hourly_v6.csv'
df_hourly = pd.read_csv(path_file_output, parse_dates=['pickup_datetime'], infer_datetime_format=True)
df_hourly['pickup_hour_year'] = df_hourly['pickup_datetime'].dt.strftime('%Y, %H hrs')
df_hourly['pickup_month_year'] = df_hourly['pickup_datetime'].dt.strftime('%B, %Y')


In [23]:

## Write to File (Ready for Modelling Datasets)
df_hourly_amount_per_year = df_hourly[['pickup_hour_year', 'total_amount']].groupby('pickup_hour_year').mean('total_amount')
df_hourly_amount_per_year = df_hourly_amount_per_year.reset_index()
df_hourly_amount_per_year.to_csv('F:\BFD Project Data\Processed\hourly_amount_per_year.csv', index=False)

df_hourly_trips_per_year = df_hourly[['pickup_hour_year', 'trips']].groupby('pickup_hour_year').mean('trips')
df_hourly_trips_per_year = df_hourly_trips_per_year.reset_index()
df_hourly_trips_per_year.to_csv('F:\BFD Project Data\Processed\hourly_trips_per_year.csv', index=False)

# df_hourly_amount_per_year_pickup = df_hourly[['pickup_hour_year', 'pickup_from', 'total_amount']]
# df_hourly_amount_per_year_pickup.to_csv('F:\BFD Project Data\Processed\hourly_amount_per_year_pickup.csv', index=False)

# df_monthly_trips_per_year = df_hourly[['pickup_month_year', 'trips']]
# df_monthly_trips_per_year.to_csv('F:\BFD Project Data\Processed\monthly_trips_per_year.csv', index=False)

In [28]:
df_hourly_amount_per_year[df_hourly_amount_per_year.pickup_hour_year.str.contains('2022')].head(30)

,pickup_hour_year,total_amount
264,"2022, 00 hrs",31.604082
265,"2022, 01 hrs",32.090944
266,"2022, 02 hrs",34.545587
267,"2022, 03 hrs",36.059998
268,"2022, 04 hrs",38.541036
269,"2022, 05 hrs",38.795948
270,"2022, 06 hrs",36.662548
271,"2022, 07 hrs",37.076387
272,"2022, 08 hrs",36.675732
273,"2022, 09 hrs",35.695913


In [54]:
df_hourly.head(10)

,pickup_datetime,payment_type,pickup_from,dropoff_at,passenger_count,trip_distance,fare_amount,tip_amount,trip_duration,total_amount,trips,congestion_category,day_category,source_file,id,pickup_hour_year,pickup_month_year
0,2011-01-01 00:00:00,Credit Card,Airport,Airport,1,5.700000,25.300000,3.940000,2808.000000,30.240000,1,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,0,"2011, 00 hrs",January 2011
1,2011-01-01 00:00:00,Credit Card,Airport,Other,1,15.244186,35.997674,6.666047,1579.302326,46.034419,43,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,1,"2011, 00 hrs",January 2011
2,2011-01-01 00:00:00,Credit Card,Other,Airport,2,10.700000,26.900000,8.170000,1559.000000,40.870000,1,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,2,"2011, 00 hrs",January 2011
3,2011-01-01 00:00:00,Credit Card,Other,Other,1,3.007332,10.813078,2.163139,848.065927,14.024267,3246,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,3,"2011, 00 hrs",January 2011
4,2011-01-01 00:00:00,Cash,Airport,Airport,1,2.233333,9.300000,0.000000,644.333333,11.823333,3,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,4,"2011, 00 hrs",January 2011
5,2011-01-01 00:00:00,Cash,Airport,Other,1,11.925000,28.457895,0.000000,1315.157895,30.030789,76,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,5,"2011, 00 hrs",January 2011
6,2011-01-01 00:00:00,Cash,Other,Airport,2,14.166667,54.733333,0.000000,1846.333333,58.733333,3,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,6,"2011, 00 hrs",January 2011
7,2011-01-01 00:00:00,Cash,Other,Other,1,2.668084,9.911324,0.004299,775.381743,10.959187,8676,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,7,"2011, 00 hrs",January 2011
8,2011-01-01 01:00:00,Cash,Other,Other,1,2.833666,10.391300,0.002915,825.151223,11.444437,9529,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,8,"2011, 01 hrs",January 2011
9,2011-01-01 01:00:00,Cash,Other,Airport,2,11.014286,28.014286,0.000000,1490.142857,30.895714,7,After Midnight Congestion,Early Morning,F:\BFD Project Data\Raw\2011\yellow_tripdata_2...,9,"2011, 01 hrs",January 2011


In [104]:
# def remove_outliers(df, column, partition_col):
#     df_result = pd.DataFrame()
#     partitions = df[partition_col].drop_duplicates().to_list()
#     for partition in partitions:
#         df_filtered = df[df[partition_col]==partition]
#         q1 = df_filtered[column].quantile(q=0.25)
#         q3 = df_filtered[column].quantile(q=0.75)
#         iqr = q3 - q1   
#         min = q1 - 1.5 * iqr
#         max = q3 + 1.5 * iqr
#         #df = df.drop(axis=0)
#         df_filtered = df_filtered[(df[partition_col]==partition) & (df[column].between(min, max))]
#         pd.concat(df_result, df_filtered)
        
#         print(partition)
#         print('Q1: {0}, Q3: {1}, IQR: {2}, Min: {3}, Max: {4}'.format(q1, q3, iqr, min, max))
#         print(df_filtered[column].describe())
#         print()
    
#     return df_filtered


### Pandas Profiler: EDA

In [3]:
profile = ProfileReport(df_hourly, title = 'Pandas Profiling Report')
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\andresespinalh\miniconda3\envs\py-data\lib\site-packages\multimethod\__init__.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return func(*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]